In [102]:
# get the top 25 stocks for each of the 49 sectors
# see how much of the returns of the sector are described by the stocks
# import the model
# run a backtest

In [103]:
# from alpha_vantage.timeseries import TimeSeries

# api_key = '3HFMMMNDMKHZXAZ3'
# ts = TimeSeries(key=api_key)
# data, meta_data = ts.get_symbol_search('IBM')
# print(data.shape)
# data

In [104]:
# from sec_edgar_api import EdgarClient

# edgar = EdgarClient(user_agent="<AAPL> <Admin Contact>@<Sample Company Domain>")

In [105]:
# import time
# start_time = time.time()
# count = 0
# while count < 100:
#     count += 1
#     edgar.get_submissions(cik='320193')

# print("--- %s submissions/seconds ---" % f'{100/(time.time() - start_time)}')

In [106]:
from sec_api import MappingApi

mappingApi = MappingApi(api_key="a403d5c05f8508aeba59443977a060e318f5feb778ff5416701f3b36e0609ef9")

nasdaq_resp = mappingApi.resolve("exchange", "NASDAQ")
# nasdaq_resp
nyse_resp = mappingApi.resolve("exchange", "NYSE")
# nyse_resp
# amex_resp = mappingApi.resolve("exchange", "AMEX")
# amex_resp

In [107]:
# filtering out stocks that are no longer listed
# filtering out stocks that are not in the US
nasdaq = [x for x in nasdaq_resp if x['isDelisted'] == False]
nyse = [x for x in nyse_resp if x['isDelisted'] == False]
# amex = [x for x in amex_resp if x['isDelisted'] == False]

In [108]:
len(nasdaq), len(nyse)
# , len(amex)

In [109]:
industries_abbr = []
industries_name = []
industries_siccodes = {}
# open the siccodes txt file
with open('siccodes49.txt') as f:
    siccodes = f.readlines()
    print(len(siccodes))

# print(siccodes[:10])

name_start_idx = 9
# col_heads, col_names = [], []

# curr_line = siccodes[0].strip()
# # print("shortened curr_line")
# # print(curr_line[:name_start_idx])
# col_head, col_name = curr_line[:name_start_idx].split(' ')[1], curr_line[name_start_idx:]

# print("col_head", col_head)
# print("col_name", col_name)
# col_heads.append(col_head)
# col_names.append(col_name)

idx = 0

while idx < len(siccodes):
    curr_line = siccodes[idx]
    industry_idx = int(curr_line[0:2].strip()) - 1
    industry_abbr = curr_line[2:10].strip()
    industry_name = curr_line[10:].strip()

    industries_abbr.append(industry_abbr)
    industries_name.append(industry_name)
    
    idx += 1

    curr_line = siccodes[idx].strip()
    while curr_line != '' and idx + 1 < len(siccodes):
        print(repr(curr_line), "in while loop")
        low  = int(curr_line[:4])
        high = int(curr_line[5:9])
        for code in range(low, high + 1):
            industries_siccodes[code] = industry_idx
        idx += 1
        curr_line = siccodes[idx].strip()
    
    if idx + 1 == len(siccodes):
        curr_line = siccodes[idx].strip()
        print(repr(curr_line), "in if statement")
        low  = int(curr_line[:4])
        high = int(curr_line[5:9])
        for code in range(low, high + 1):
            industries_siccodes[code] = industry_idx
        break 

    idx += 1
    
    print("industry_idx", industry_idx)
    print("industry_abbr", industry_abbr)
    print("industry_name", industry_name)
    

print(industries_abbr)
print(industries_name)
print(industries_siccodes)

In [110]:
nasdaq[0]

In [111]:
nasdaq

In [112]:
nyse[0]

In [113]:
# for each stock, check if the siccode is in the set of siccodes for the
# industry
# if it is, add it to the list of stocks for that industry

industry_not_found = 0

industry_stocks = {}
for abbr in industries_abbr:
    industry_stocks[abbr] = {}

for idx, stock in enumerate(nasdaq):
    try:
        siccode = int(stock['sic'])
    except ValueError:
        print(f"could not convert siccode to int for stock {stock['ticker']} (index: {idx})")
        industry_not_found += 1
        continue

    industry = industries_siccodes.get(siccode, None)

    if industry is None:
        print(f"industry not found for stock {stock['ticker']} (index: {idx})")
        industry_not_found += 1
        continue
    
    print("=====")
    print(industry)
    print(industries_abbr[industry])
    industry_stocks[industries_abbr[industry]][stock['ticker']] = stock

print(f"Was not able to match industries for {industry_not_found} stocks")

In [114]:
industry_stocks['Chips']['NVDA']

In [117]:
import yfinance as yf
import pandas as pd

final_ind_stocks = { ind: {} for ind in industries_abbr }

for abbr in industries_abbr:
    print("=============================================================")
    print(f"Industry: {abbr}")

    for ticker, stock_info in industry_stocks[abbr].items():
        # industry_stocks[abbr][stock]['market_cap']

        # get the historical data for each stock
        yf_data = yf.Ticker(ticker)
        fama_french_calc_date = '2023-06-29'

        hist = yf_data.history(start='2023-06-29', end='2023-06-30', period='1d')
        if hist.empty:
            print(f"No historical data available for {ticker}")
            continue
        hist.index = hist.index.strftime('%Y-%m-%d')

        # Ensure there is data for the specific date
        if fama_french_calc_date in hist.index:
            # Get the closing price on the specific date
            close_price = hist.loc[fama_french_calc_date, 'Close']

            # Get the number of shares outstanding
            shares_outstanding = yf_data.info.get('sharesOutstanding')
            
            if shares_outstanding:
                # Calculate the market cap
                market_cap = close_price * shares_outstanding
                final_ind_stocks[abbr][ticker] = {'mkt_cap' : market_cap, 'stock_info' : stock_info}
                # print(f"Market cap of {ticker} on {fama_french_calc_date}: ${market_cap:,.2f}")
            else:
                print(f"Shares outstanding information is not available for {ticker}.")
        else:
            print(f"No data available for {fama_french_calc_date} for {ticker}")

    print()

In [118]:
total_stocks = 0
found_stocks = 0
for abbr in industries_abbr:
    print(f'There are {len(industry_stocks[abbr])} stocks (...and {len(final_ind_stocks[abbr])} found) in the {abbr} industry')
    total_stocks += len(industry_stocks[abbr])
    found_stocks += len(final_ind_stocks[abbr])

print(f'Total number of stocks: {total_stocks}')
print(f'Total number of stocks found: {found_stocks}')

In [ ]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol
ticker_symbol = 'AAGR'

# Fetch the stock data
stock = yf.Ticker(ticker_symbol)

try:
    # Try fetching historical data for the last 5 years
    historical_data = stock.history(period='5y', interval="1d")

    # Check if historical data is available
    if not historical_data.empty:
        # Get the earliest available date
        earliest_date = historical_data.index.min()
        print(f"The earliest available data for {ticker_symbol} is from {earliest_date.date()}")
    else:
        print(f"No historical data available for {ticker_symbol}")
except Exception as e:
    print(f"Error fetching historical data for {ticker_symbol}: {e}")

In [ ]:
final_ind_stocks

In [127]:
import os

stocks_dir = 'industry_stocks'

# save the final_ind_stocks to a csv
for ind in final_ind_stocks:
    try: 
        df = pd.DataFrame.from_dict(final_ind_stocks[ind], orient='index')
        # sort the dataframe by highest mkt_cap
        print(f"Industry: {ind}")
        df = df.sort_values(by='mkt_cap', ascending=False)
        # display(df)
        
        os.makedirs(stocks_dir, exist_ok=True)
        df.to_csv(f'{stocks_dir}/{ind}_stocks.csv')
    except Exception as e:
        print(f"Error saving {ind} stocks: {e}")
        continue

In [ ]:
# import yfinance as yf
# import pandas as pd

# # get the historical data for each stock
# ticker_symbol = 'AAPL'
# stock = yf.Ticker("AAPL")
# fama_french_calc_date = '2023-06-29'

# hist = stock.history(start='2023-06-29', end='2023-06-30', interval='1d')
# hist.index = hist.index.strftime('%Y-%m-%d')

# # Ensure there is data for the specific date
# if fama_french_calc_date in hist.index:
#     # Get the closing price on the specific date
#     close_price = hist.loc[fama_french_calc_date, 'Close']

#     # Get the number of shares outstanding
#     shares_outstanding = stock.info.get('sharesOutstanding')
    
#     if shares_outstanding:
#         # Calculate the market cap
#         market_cap = close_price * shares_outstanding
#         print(f"Market cap of {ticker_symbol} on {fama_french_calc_date}: ${market_cap:,.2f}")
#     else:
#         print("Shares outstanding information is not available.")
# else:
#     print(f"No data available for {fama_french_calc_date}")

In [ ]:
import requests

# Define the endpoint and parameters for the SEC API
api_endpoint = "https://data.sec.gov/api/v1/tickers"

# Set the headers
headers = {
    "User-Agent": "Your Name (your.email@example.com)"
}

# Make the request to the SEC API
response = requests.get(api_endpoint, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Filter the data for AMEX exchange
    amex_stocks = [item for item in data if item.get("exchange") == "AMEX"]

    # Print the list of AMEX stocks
    for stock in amex_stocks:
        print(f"Ticker: {stock.get('ticker')}, Name: {stock.get('name')}")
else:
    print(f"Failed to fetch data: {response.status_code}")